# Encoder Feed Forward

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_encoder_feed_forward.png" alt="Encoder Feed Forward" style="width:425px;height:626px;">
</div>

A continuación viene una capa que llaman `Feed Forward`, que consiste en una red fully conected como las que hemos visto al principio del curso, pero de dos capas, exactamente en el paper lo definen así

<div style="text-align:center;">
  <img src="Imagenes/feed_forward_equation.png" alt="Feed Forward equation">
</div>

Esta fórmula lo que nos indica es que hay que pasar la matriz por una capa lineal con pesos $W_1$ y $b_1$, aplicarle una función de activación Relu y a continuación pasarle por otra capa lineal con pesos $W_2$ y $b_2$

El motivo de esta capa es añadir una no linealidad al encoder, lo cual trae las siguientes ventajas

 * Incremento de la capacidad del modelo: Permite que el modelo aprenda representaciones más complejas. Si solo tuviéramos operaciones lineales, la capacidad del modelo para aprender relaciones complejas en los datos se vería significativamente limitada como ya hemos estudiado

 * Extracción de características: Aunque las capas de atención de los Transformers son buenas para capturar relaciones a largo plazo entre palabras en un texto, la capa Feed Forward puede ayudar a extraer características más locales de cada palabra, de manera similar a cómo las capas densas en una red neuronal convencional pueden aprender a extraer características de sus entradas.

Como se puede ver, después de la capa `Feed Forward` va a haber una capa de `Add and Norm` lo cual quiere decir que a esta capa `Add & Norm` se va a sumar la matriz que entre a la capa `Feez Forward` y la que salga de esta, por lo que para poder sumar las dos matrices tienen que ser de las mismas dimensiones.

<div style="text-align:center;">
  <img src="Imagenes/transformer_architecture_model_encoder_feed_forward.png" alt="Encoder Feed Forward" style="width:425px;height:626px;">
</div>

Es decir, que la capa `Feed Forward` va a tener que tener unos pesos $W_2$ y $b_2$ que hagan que a la salida de la capa `Feed Forward` se mantenga la dimensión

Sin embargo, a la salida de la primera capa de la red `Feed Forward` da igual las dimensiones de la matriz resultante.

En el paper proponen que si la matriz de entrada tiene una dimensión de embedding de 512, a la salida de la primera capa la matriz tenga una dimensión de embedding de 2048 y a la salida de la segunda capa vuelva a tener una dimensión de embedding de 512. Por lo que en la parte de `Feed Forward` internamente se multiplica la dimensión de la matriz por 4 y luego se vuelve a reducir

## Implementación

Vamos a crear la clase de la capa `Feed Forward`

In [1]:
from torch import nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, dim_embedding, increment=4):
        super().__init__()
        self.feed_forward = nn.Sequential(
            nn.Linear(dim_embedding, dim_embedding*increment),
            nn.ReLU(),
            nn.Linear(dim_embedding*increment, dim_embedding)
        )
    
    def forward(self, x):
        """
        Args:
            x (torch.Tensor): (batch_size, seq_len, dim_embedding)

        Returns:
            torch.Tensor: (batch_size, seq_len, dim_embedding)
        """
        x = self.feed_forward(x)
        return x

Vamos a coger el embbeding preentrenado de BERT, las implementaciones anteriores que hemos hecho antes y vamos a ver qué obtenemos a la salida de nuestra clase

In [5]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer

class ScaledDotProductAttention(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding: dimension of embedding vector
        """
        super().__init__()
        self.dim_embedding = dim_embedding
    
    def forward(self, key, query, value):
        """
        Args:
            key: key vector
            query: query vector
            value: value vector
        
        Returns:
            output vector from scaled dot product attention
        """
        # MatMul
        key_trasposed = key.transpose(-1,-2)
        product = torch.matmul(query, key_trasposed)
        # scale
        scale = product / torch.sqrt(torch.tensor(self.dim_embedding))
        # softmax
        attention_matrix = torch.softmax(scale, dim=-1)
        # MatMul
        output = torch.matmul(attention_matrix, value)
        
        return output

class MultiHeadAttention(nn.Module):
    def __init__(self, heads, dim_embedding):
        super().__init__()
        
        self.dim_embedding = dim_embedding
        self.dim_proyection = dim_embedding // heads
        self.heads = heads
        
        self.proyection_Q = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_K = nn.Linear(dim_embedding, dim_embedding)
        self.proyection_V = nn.Linear(dim_embedding, dim_embedding)
        self.attention = nn.Linear(dim_embedding, dim_embedding)

        self.scaled_dot_product_attention = ScaledDotProductAttention(self.dim_proyection)
    
    def forward(self, Q, K, V):
        batch_size = Q.size(0)
        
        # perform linear operation and split into h heads
        proyection_Q = self.proyection_Q(Q).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_K = self.proyection_K(K).view(batch_size, -1, self.heads, self.dim_proyection)
        proyection_V = self.proyection_V(V).view(batch_size, -1, self.heads, self.dim_proyection)
        
        # transpose to get dimensions bs * h * sl * d_model
        proyection_Q = proyection_Q.transpose(1,2)
        proyection_K = proyection_K.transpose(1,2)
        proyection_V = proyection_V.transpose(1,2)

        # calculate attention
        scaled_dot_product_attention = self.scaled_dot_product_attention(proyection_Q, proyection_K, proyection_V)
        
        # concatenate heads and put through final linear layer
        concat = scaled_dot_product_attention.transpose(1,2).contiguous().view(batch_size, -1, self.dim_embedding)
        
        output = self.attention(concat)
    
        return output

class AddAndNorm(nn.Module):
    def __init__(self, dim_embedding):
        """
        Args:
            dim_embedding (int): Embedding dimension.
        """
        super().__init__()
        self.normalization = nn.LayerNorm(dim_embedding)

    def forward(self, x, sublayer):
        """
        Args:
            x (torch.Tensor): Input tensor.
            sublayer (torch.Tensor): Sublayer tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        return self.normalization(torch.add(x, sublayer))

def extract_embeddings(input_sentences, model_name='bert-base-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)
    
    # tokenización de lote
    inputs = tokenizer(input_sentences, return_tensors='pt', padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    token_embeddings = outputs[0]
    
    # Los embeddings posicionales están en la segunda capa de los embeddings de la arquitectura BERT
    positional_encodings = model.embeddings.position_embeddings.weight[:token_embeddings.shape[1], :].detach().unsqueeze(0).repeat(token_embeddings.shape[0], 1, 1)

    embeddings_with_positional_encoding = token_embeddings + positional_encodings

    # convierte las IDs de los tokens a tokens
    tokens = [tokenizer.convert_ids_to_tokens(input_id) for input_id in inputs['input_ids']]

    return tokens, inputs['input_ids'], token_embeddings, positional_encodings, embeddings_with_positional_encoding

Obtenemos el resultado del input embedding más el positional encoding

In [6]:
sentence1 = "I gave the dog a bone because it was hungry"
tokens1, input_ids1, token_embeddings1, positional_encodings1, embeddings_with_positional_encoding1 = extract_embeddings(sentence1)

Instanciamos un objeto de la clase `Multi-Head Attention` y obtenemos su salida

In [7]:
dim_embedding = embeddings_with_positional_encoding1.shape[-1]
heads = 8
multi_head_attention = MultiHeadAttention(heads=heads, dim_embedding=dim_embedding)
attention = multi_head_attention(embeddings_with_positional_encoding1, embeddings_with_positional_encoding1, embeddings_with_positional_encoding1)

Ahora instanciamos un objeto de la clase `Add & Norm`

In [8]:
add_and_norm = AddAndNorm(dim_embedding=dim_embedding)
add_and_norm_output = add_and_norm(embeddings_with_positional_encoding1, attention)

E instanciamos un objeto de la clase `Feed Forward`

In [9]:
feed_forward = FeedForward(dim_embedding=dim_embedding)

Metemos la salida del anterior `Add & Norm` en la clase `Feed Forward`

In [10]:
feed_forward_output = feed_forward(add_and_norm_output)
feed_forward_output.shape

torch.Size([1, 12, 768])

Seguimos obteniendo una matriz de 1x12x768, parece que todo bien